In [ ]:
%pip install db-sqlite3 # Add the library to Python
%pip install pandas
%pip install matplotlib

In [1]:
# Imports 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt

# Input data files are available in the "../data/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

path = "/Users/wichorivera/Downloads/data-analyst-interview-master/data/artists.db"  #Insert path here
database = '/Users/wichorivera/Downloads/data-analyst-interview-master/data/artists.db' 

Create Database connection:

In [2]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables 

# Connect the master database that contain the two tables artist & artworks

,type,name,tbl_name,rootpage,sql
0,table,artists,artists,2,"CREATE TABLE ""artists"" (\n""Artist ID"" INTEGER,..."
1,table,artworks,artworks,159,"CREATE TABLE ""artworks"" (\n""Artwork ID"" INTEGE..."


If you can see the 'tables' object above, then you're good to go!

In [3]:
# Check numbers of records in the tables: artworks 130,262 records ; artists 15,091 records

artists = pd.read_sql("""SELECT COUNT(*) as "Artist Records"
                        FROM artists
                        ;""", conn)
artists

,Artist Records
0,15091


In [4]:
# Artworks check records matching

artists = pd.read_sql("""SELECT COUNT(*) as "Artworks Records"
                        FROM artworks
                        ;""", conn)
artists

,Artworks Records
0,130262


In [5]:
# Question 1: Which artist in this data set lived the longest? 

artists = pd.read_sql("""SELECT ar.Name,
                        ar."Death Year"-ar."Birth Year" as YearsLived           
                        FROM artists ar
                        WHERE YearsLived>=1
                        ORDER BY YearsLived DESC
                        LIMIT 3
                        ;""", conn)
artists

# Ensure with this filter Death Year - Birth Year >= 1
# The top 1 in this query is a company so the second & third rows as artist with 108 years (Leni Matthaei, B. Efimov)

,Name,YearsLived
0,"Union Paper Bag Machine Company, Philadelphia, PA",130
1,Leni Matthaei,108
2,B. Efimov,108


In [6]:
# Test for the artist still alive and reach 108 years

artists = pd.read_sql("""SELECT T1.Name, T1.ActualYear-T1."Birth Year" AS YearsLived
                FROM (SELECT ar.Name, ar."Birth Year",
                        COALESCE(ar."Death Year","2022") as ActualYear
                            FROM artists ar
                        WHERE ar."Birth Year" is not null
                    )T1
                        ORDER BY YearsLived desc
                        limit 5
                        ;""", conn)
artists

# Reason non-viable solution: Cannot assure who still alive & which has missing values, the top years lived is 291

,Name,YearsLived
0,"J.A. Henckels, Solingen, Germany",291
1,"Compagnie des Cristalleries de Baccarat, Paris",257
2,Anton Schranz II,221
3,"Scovill Mfg. Co., Plumbers Brass Goods Div., W...",220
4,"International Silver Co., Meriden, CT",214


In [7]:
# Question 2: Who are the top 10 artists by the number of artworks?

artists = pd.read_sql("""SELECT ar.Name, 
                        COUNT(aw."Artwork ID") as Num_Artworks       
                        FROM artists ar
                        INNER JOIN artworks aw on ar."Artist ID"=aw."Artist ID"
                        GROUP BY 1
                        ORDER BY Num_Artworks DESC
                        LIMIT 10
                        ;""", conn)
artists

# Inner join to ensure all the artist listed shows the number of the artworks and count per artwork id 

,Name,Num_Artworks
0,Eugène Atget,5050
1,Louise Bourgeois,3318
2,Ludwig Mies van der Rohe,2566
3,Unknown photographer,1575
4,Jean Dubuffet,1435
5,Lee Friedlander,1317
6,Pablo Picasso,1310
7,Marc Chagall,1162
8,Henri Matisse,1063
9,Pierre Bonnard,894


In [8]:
# Test for the second solution option with only artwork table

artists = pd.read_sql("""SELECT aW.Name,aw."Artist ID",COUNT(aw."Artwork ID") AS ARTS       
                        FROM artworks aw 
                        GROUP BY 1,2
                        ORDER BY ARTS DESC
                        LIMIT 10
                        ;""", conn)
artists

# Reason non-viable solution: the artworks that had no artist linked (none) but appearing in the count

,Name,Artist ID,ARTS
0,Eugène Atget,229,5050
1,Louise Bourgeois,710,3318
2,Ludwig Mies van der Rohe,7166,2566
3,Unknown photographer,8595,1575
4,None,None,1460
5,Jean Dubuffet,1633,1435
6,Lee Friedlander,2002,1317
7,Pablo Picasso,4609,1310
8,Marc Chagall,1055,1162
9,Henri Matisse,3832,1063


In [9]:
# Question 3: Which artist is created the most artwork by total surface area?

artists = pd.read_sql(""" SELECT Name,
            COALESCE(Circle,Tria,square) as Max
FROM (
SELECT  Name,
case when "Diameter (cm)" not null 
    and "Height (cm)" is null and "Length (cm)" is null and "Width (cm)" is null
then (3.1415*(("Diameter (cm)"/12)*("Diameter (cm)"/12))) else 0 end as Circle,

case when "Diameter (cm)" is null 
    and "Height (cm)" not null and "Length (cm)"not null and "Width (cm)" is null
then ("Height (cm)"*"Length (cm)")/2 else 0 end as Tria,
    
case when "Diameter (cm)" is null 
    and "Height (cm)" is null and "Length (cm)"not null and "Width (cm)" is not null
then "Width (cm)"*"Length (cm)" else 0 end as square

                        FROM artworks aw 
     ORDER BY 2 DESC)t1          
                       limit 1
                        ;""", conn)
artists

# Disclaimer: I could assume that all the artworks are squares and rectangles and just multiply width and length
# I only asume 3 figures to get the surface area, could be more

,Name,Max
0,Richard Long,7838.050354
1,Polly Apfelbaum,6566.761586


In [10]:
# Question 4: Did any artists have artwork acquired during their lifetime?

artists = pd.read_sql(""" SELECT count(distinct t1."Artist ID") as Artist_Num
    FROM(SELECT ar."Artist ID",
        CASE WHEN (trim(aw.Date,"ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz_-., ()"))<=ar."Death Year"
        THEN 1 ELSE 0 END AS Acquired
                        FROM artists ar
                        INNER JOIN artworks aw on ar."Artist ID"=aw."Artist ID"
                        WHERE ar."Birth Year" is not null
                        AND ar."Death Year" is not null 
                        AND aw.Date is not null) t1
                WHERE t1.Acquired=1
                
                        ;""", conn)
artists

# The trim was added for cleaning date field and it has a subquery for pulling out the ones that not accomplish
# Clauses "is not null" ensure no missing data and last count distinct avoiding duplicate artist

,Artist_Num
0,3878


In [11]:
# Question 6: Please group the artworks into as many clusters as you feel is appropriate, using attributes 
# from both the artist and artworks tables, and assign each artwork to this new cluster.

artists = pd.read_sql("""SELECT ar.Nationality, 
                        COUNT(aw."Artwork ID") as Num_Artworks,
                    (COUNT(aw."Artwork ID")/cast((select count("Artwork ID")from artworks) as float))*100 as Per_Total
                        FROM artists ar
                        INNER JOIN artworks aw on ar."Artist ID"=aw."Artist ID"
                        GROUP BY 1
                        ORDER BY Num_Artworks Desc
                        limit 10
                    """, conn)
artists

# By nationality

,Nationality,Num_Artworks,Per_Total
0,American,54545,41.873302
1,French,22303,17.121647
2,German,8987,6.899172
3,British,5500,4.222260
4,None,3310,2.541033
5,Spanish,2954,2.267737
6,Italian,2719,2.087332
7,Japanese,2355,1.807895
8,Swiss,1948,1.495448
9,Russian,1473,1.130798


In [12]:
#By Gender

artists = pd.read_sql("""SELECT case when ar.Gender='male' then 'Male' else ar.Gender end as Gender,
                        COUNT(aw."Artwork ID") as Num_Artworks,
                    (COUNT(aw."Artwork ID")/cast((select count("Artwork ID")from artworks) as float))*100 as Per_Total
                        FROM artists ar
                        INNER JOIN artworks aw on ar."Artist ID"=aw."Artist ID"
                        GROUP BY 1
                        ORDER BY Num_Artworks Desc
                    """, conn)
artists

,Gender,Num_Artworks,Per_Total
0,Male,99984,76.756076
1,Female,16193,12.431100
2,None,6196,4.756568


In [13]:
#By Catalogue

artists = pd.read_sql("""SELECT aw.Catalogue,
                        COUNT(aw."Artwork ID") as Num_Artworks,
                    (COUNT(aw."Artwork ID")/cast((select count("Artwork ID")from artworks) as float))*100 as Per_Total
                        FROM artists ar
                        INNER JOIN artworks aw on ar."Artist ID"=aw."Artist ID"
                        GROUP BY 1
                        ORDER BY Num_Artworks Desc
                    """, conn)
artists

,Catalogue,Num_Artworks,Per_Total
0,Y,66951,51.397184
1,N,55422,42.546560


In [14]:
#By Classification

artists = pd.read_sql("""SELECT aw.Classification,
                        COUNT(aw."Artwork ID") as Num_Artworks,
                    (COUNT(aw."Artwork ID")/cast((select count("Artwork ID")from artworks) as float))*100 as Per_Total
                        FROM artists ar
                        INNER JOIN artworks aw on ar."Artist ID"=aw."Artist ID"
                        GROUP BY 1
                        ORDER BY Num_Artworks Desc
                        limit 5
                    """, conn)
artists

,Classification,Num_Artworks,Per_Total
0,Print,29786,22.866223
1,Photograph,28400,21.802214
2,Illustrated Book,25326,19.442355
3,Drawing,11509,8.835270
4,Design,9793,7.517925


In [ ]:
#By Longevity - how long the artwork has been acquired and still in catalogue

artists = pd.read_sql("""
                        SELECT DATE('2022-09-13')-aw."Acquisition Date" AS Years,
                        COUNT(aw."Artwork ID") as Num_Artworks
                        FROM artists ar
                        INNER JOIN artworks aw on ar."Artist ID"=aw."Artist ID"
                        WHERE Catalogue = 'Y' AND aw."Acquisition Date" IS NOT NULL
                        GROUP BY 1
                        ORDER BY Num_Artworks Desc
                        limit 6
                    """, conn)
artists

In [15]:
#By all possible combinations 

artists = pd.read_sql("""SELECT  aw.Classification,ar.Nationality,aw.Catalogue,
                        case when ar.Gender='male' then 'Male' else ar.Gender end as Gender,
                        COUNT(aw."Artwork ID") as Num_Artworks,
                    (COUNT(aw."Artwork ID")/cast((select count("Artwork ID")from artworks) as float))*100 as Per_Total
                        FROM artists ar
                        INNER JOIN artworks aw on ar."Artist ID"=aw."Artist ID"
                        GROUP BY 1,2,3,4
                        ORDER BY Num_Artworks Desc
                        limit 6
                    """, conn)
artists

# storytelling: we 5% of the artworks acquired and still in place are prints from american males

,Classification,Nationality,Catalogue,Gender,Num_Artworks,Per_Total
0,Illustrated Book,French,N,Male,9932,7.624633
1,Print,American,Y,Male,6617,5.079762
2,Photograph,American,N,Male,6433,4.938509
3,Photograph,American,Y,Male,5160,3.961247
4,Drawing,American,Y,Male,4262,3.271867
5,Print,American,N,Male,4214,3.235019
